### Target Webscraping

Using Selenium again to go through all the data in Target and see what we can get

Source list
+ https://www.youtube.com/watch?v=SPM1tm2ZdK4&t=855s
+ https://www.youtube.com/watch?v=USrjHgO9Niw python selenium button clicking


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import csv
import re
import pandas as pd 

grocery_list = pd.read_csv('Grocery List.csv')

college_file_list = ['ames, ia', 'college station, tx'] 
# with open('Collegetowns2.csv', 'r') as file:
#     # Create a CSV reader object using DictReader
#     for line in file:
#         city = line[:-1].replace(',', '-').replace(' ', '-')
#         college_file_list.append(city)

options=Options()
options.add_experimental_option("detach", True)

In [2]:
grocery_df = grocery_list.T
grocery_df.columns = grocery_df.iloc[0]
grocery_df = grocery_df[1:]
grocery_df = grocery_df.reset_index(drop=True)
grocery_df.columns

food_prices = pd.DataFrame(college_file_list, columns=["Location"])
food_prices = pd.concat([food_prices, grocery_df], axis=1)
food_prices

,Location,Dark Red Kidney Beans,Ground Beef x 2,1/2 gallon milk,Cereal,Lunch meat,Bread,American cheese,Ramen pack,Poptarts,...,Crushed Tomatoes,Light Red Kidney Beans,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4
0,"ames, ia",https://www.target.com/p/bush-39-s-dark-red-ki...,https://www.target.com/p/all-natural-85-15-gro...,https://www.target.com/p/2-reduced-fat-milk-0-...,https://www.target.com/p/cheerios-breakfast-ce...,https://www.target.com/p/uncured-honey-ham-ult...,https://www.target.com/p/sara-lee-whole-grain-...,https://www.target.com/p/kraft-singles-america...,https://www.target.com/p/maruchan-souper-6-pac...,https://www.target.com/p/pop-tarts-frosted-bro...,...,https://www.target.com/p/muir-glen-organic-glu...,https://www.target.com/p/organic-low-sodium-li...,https://www.target.com/p/barilla-spaghetti-pas...,https://www.target.com/p/tomato-basil-38-garli...,https://www.target.com/p/green-bell-peppers-2c...,https://www.target.com/p/tomato-paste-6oz-good...,https://www.target.com/p/yellow-onion-each/-/A...,https://www.target.com/p/100-apple-juice-64-fl...,https://www.target.com/p/land-o-lakes-unsalted...,https://www.target.com/p/folgers-classic-mediu...
1,"college station, tx",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                            options=options)

for index, row in grocery_list.iterrows():
    print("Starting on: ", row["Item"])
    
    driver.get(row["URL"])
    time.sleep(3)
    for college in college_file_list:

        # Find the button by its ID
        webStoreButton = driver.find_element(By.ID, "web-store-id-msg-btn")

        # Click the button
        webStoreButton.click()
        time.sleep(3)

        inputBox = driver.find_element(By.ID, "zip-or-city-state")
        inputBox.send_keys(college) # Need to input "city, st"

        lookupButton =  driver.find_element(By.CSS_SELECTOR, '[data-test="@web/StoreLocationSearch/button"]')
        lookupButton.click()
        time.sleep(3)

        ShopStoreButton =  driver.find_element(By.CSS_SELECTOR, '[data-test="@web/StoreMenu/ShopThisStoreButton"]')
        ShopStoreButton.click()

        time.sleep(3)
        # ID of the button id="zip-code-id-btn"
        
        # Putting the value in last so that we can ensure on the first and last iteration it will be in the right spot
        price_element = driver.find_element(By.CSS_SELECTOR, '[data-test="product-price"]')
        food_prices.loc[food_prices['Location'] == college, row['Item']] = price_element.text
        time.sleep(3)


driver.close()

Starting on:  Dark Red Kidney Beans
Starting on:  Ground Beef x 2
Starting on:  1/2 gallon milk
Starting on:  Cereal
Starting on:  Lunch meat
Starting on:  Bread
Starting on:  American cheese
Starting on:  Ramen pack
Starting on:  Poptarts
Starting on:  Frozen pizza
Starting on:  Eggs Dozen
Starting on:  Potato
Starting on:  Celsius 4 pack
Starting on:  Crushed Tomatoes
Starting on:  Light Red Kidney Beans
Starting on:  Spaghetti Noodles
Starting on:  Spaghetti Sauce
Starting on:  Green Peppers
Starting on:  Tomato Paste
Starting on:  Onions x2
Starting on:  Apple Juice
Starting on:  Butter 1/4
Starting on:  Coffee 1/4


In [6]:
food_prices

,Location,Dark Red Kidney Beans,Ground Beef x 2,1/2 gallon milk,Cereal,Lunch meat,Bread,American cheese,Ramen pack,Poptarts,...,Crushed Tomatoes,Light Red Kidney Beans,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4
0,"ames, ia",$1.29,$6.49,$1.69,$5.29,$3.49,$3.29,$4.29,$1.99,$3.99,...,$3.19,$1.29,$1.89,$1.69,$1.89,$0.85,$0.99,$1.99,$5.79,$13.29
1,"college station, tx",$1.29,$6.49,$1.59,$5.29,$3.49,$3.29,$4.29,$1.99,$3.99,...,$2.49,$1.29,$1.89,$1.69,$1.59,$0.85,$1.19,$1.99,$5.79,$13.29


In [7]:
columns_to_convert = food_prices.columns
columns_to_convert = columns_to_convert[1:]
columns_to_convert

food_prices2 = food_prices.copy()
food_prices2[columns_to_convert] = food_prices2[columns_to_convert].applymap(lambda x: pd.to_numeric(x.replace('$', '')))

# These sums are somehow off
row_sum = food_prices2[columns_to_convert].sum(axis=1)
food_prices2['Row Sum'] = round(row_sum,2)
food_prices2

,Location,Dark Red Kidney Beans,Ground Beef x 2,1/2 gallon milk,Cereal,Lunch meat,Bread,American cheese,Ramen pack,Poptarts,...,Light Red Kidney Beans,Spaghetti Noodles,Spaghetti Sauce,Green Peppers,Tomato Paste,Onions x2,Apple Juice,Butter 1/4,Coffee 1/4,Row Sum
0,"ames, ia",1.29,6.49,1.69,5.29,3.49,3.29,4.29,1.99,3.99,...,1.29,1.89,1.69,1.89,0.85,0.99,1.99,5.79,13.29,79.79
1,"college station, tx",1.29,6.49,1.59,5.29,3.49,3.29,4.29,1.99,3.99,...,1.29,1.89,1.69,1.59,0.85,1.19,1.99,5.79,13.29,79.43


In [9]:
food_prices2.to_csv("College Food Prices.csv", index=False)